In [4]:
import seaborn as sb
import sys  
import pandas as pd
import requests
import numpy as np
import transliterate
import datetime
import time
import pickle
import matplotlib.pyplot as plt
import time
%matplotlib inline

from bs4 import BeautifulSoup as bs
import urllib

In [5]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [6]:
app_id = '6735892'
access_token = '3c4afdb93c4afdb93c4afdb9923c2c35ad33c4a3c4afdb9679dad4e5f90156bab3ea737'

In [7]:
tj_id = 28261334

In [9]:
my_id = 'pronpavel'

# api call and test
def vk_get_response(method, parameters, token):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + token
    return(requests.get(url).json())

answer = vk_get_response(
    'users.get', 'user_ids={0}&v=4.9&lang=ru'.format(my_id), access_token
)['response']

print(answer[0]['first_name'], answer[0]['last_name'])

Павел Пронченко


In [10]:
check_group_id = tj_id * -1
check = vk_get_response(
        'wall.get', 'owner_id={0}&count=50&fields=post_type,marked_as_ads&v=4.9'.format(
            int(check_group_id)
        ),
    access_token
)

In [11]:
check

{'response': [59156,
  {'id': 906525,
   'from_id': -28261334,
   'to_id': -28261334,
   'date': 1540837679,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': '17-летний Артём Кирсанов из Красноярска стал первым в России, кто попал в финал научного конкурса от Марка Цукерберга и Сергея Брина. Главный приз — это грант на обучение в любом университете мира.<br><br>В интервью TJ он рассказал, как взялся за тему, за которую в 2017 году дали Нобелевскую премию, а также почему в российских школах не нужны оценки и обязательное домашнее задание.',
   'media': {'type': 'share',
    'share_url': 'https://tjournal.ru/79054-ucheniki-ne-dolzhny-sidet-molcha-intervyu-krasnoyarskogo-shkolnika-kotoryy-pretenduet-na-grant-ot-cukerberga-i-brina',
    'share_title': '«Ученики не должны сидеть молча»: интервью красноярского школьника, который претендует на грант от Цукерберга и Брина'},
   'attachments': [{'type': 'link',
     'link': {'url': 'https://tjournal.ru/79054-ucheniki-ne-dolzhny-sidet-m

In [20]:
users = {pronpavel} #load_obj('final_data_user_only')
uids = [uid[0].get('uid') for uid in users]

NameError: name 'pronpavel' is not defined

In [15]:
print_counter = 0
final_data = []

for uid in uids[:1000]:
    try
        user_subs = vk_get_response(
            'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
        )
        time.sleep(0.3)
        final_data.append(user_subs)
    except:
        print("Error")
    if print_counter % 100 == 0:
        print("{0} profiles done".format(print_counter))
    print_counter += 1

NameError: name 'uids' is not defined

In [77]:
subs_list = []
groups_freq_dict = {}
top_n = 5

for record, uid in zip(final_data, uids):
    try:
        user_subs = record
        if not user_subs.get('response'):
            user_subs = vk_get_response(
                'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
            )
        subs_pd = pd.DataFrame(
            [
                {
                    'groups_count': user_subs['response']['groups'].get('count'),
                    'groups_list': user_subs['response']['groups'].get('items'),
                    'follows_count':user_subs['response']['users'].get('count'),
                    'follows_list': user_subs['response']['users'].get('items'),
                }
            ]
        )

        for group_id in user_subs['response']['groups'].get('items')[:top_n]:
            if groups_freq_dict.get(group_id):
                groups_freq_dict[group_id] += 1
            else:
                groups_freq_dict[group_id] = 1

        subs_pd['subs_count'] = subs_pd['groups_count'] + subs_pd['follows_count']
        subs_list.append(subs_pd)
    except:
#         print(user_subs)
        pass
    if len(subs_list) % 100 == 0:
        print("Processed {0} users".format(len(subs_list)))

Processed 100 users
Processed 200 users
Processed 200 users
Processed 300 users
Processed 300 users
Processed 400 users
Processed 500 users
Processed 600 users
Processed 700 users
Processed 800 users


In [82]:
sorted([(key, val) for key, val in groups_freq_dict.items()], key=lambda x: x[1], reverse=True)[:5]

[(84354128, 51),
 (57846937, 33),
 (29246653, 32),
 (40498005, 30),
 (28528027, 25)]

In [83]:
group_doc_dict = {}
counter = 0
groups_freq_dict_top5 = groups_freq_dict

for group_id, freq in groups_freq_dict_top5.items():
    counter += 1
    try:
        check = vk_get_response(
            'wall.get',
            'owner_id={0}&count=100&fields=post_type,marked_as_ads&&v=4.9&lang=ru'.format(int(group_id) * -1),
            access_token_3
        )
        check = check['response']
        group_doc = ''
        if check[0] != 0:
            for post in check[1:]:
                if post.get('marked_as_ads') != 1:
                    group_doc += post['text']
        group_doc_dict[group_id] = group_doc
    except:
        print("Response error. Group id {0}".format(group_id))
        print(check)
    if counter % 100 == 0:
        print("{0} groups extracted".format(counter))
    time.sleep(0.3)

100 groups extracted
200 groups extracted
300 groups extracted
400 groups extracted
500 groups extracted
600 groups extracted
700 groups extracted
800 groups extracted
900 groups extracted
1000 groups extracted
1100 groups extracted
1200 groups extracted
1300 groups extracted
1400 groups extracted
1500 groups extracted
1600 groups extracted
1700 groups extracted
1800 groups extracted
1900 groups extracted
2000 groups extracted
2100 groups extracted
2200 groups extracted
2300 groups extracted


In [87]:
save_obj(group_doc_dict, 'group_doc_dict')

In [13]:
from bs4 import BeautifulSoup
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from pymorphy2 import MorphAnalyzer

In [ ]:
chrs_to_delete = string.punctuation + u'»' + u'«' + u'—' + u'“' + u'„' + u'•' + u'#'
translation_table = {ord(c): None for c in chrs_to_delete if c != u'-'}
units = MorphAnalyzer.DEFAULT_UNITS
morph = MorphAnalyzer(result_type=None, units=units)
PortSt = PorterStemmer()
stopw = set(
    [w for w in stopwords.words(['russian', 'english'])]
    + [u'это', u'году', u'года', u'также', u'етот',
       u'которые', u'который', u'которая', u'поэтому',
       u'весь', u'свой', u'мочь', u'eтот', u'например',
       u'какой-то', u'кто-то', u'самый', u'очень', u'несколько',
       u'источник', u'стать', u'время', u'пока', u'однако',
       u'около', u'немного', u'кроме', u'гораздо', u'каждый',
       u'первый', u'вполне', u'из-за', u'из-под',
       u'второй', u'нужно', u'нужный', u'просто', u'большой',
       u'хороший', u'хотеть', u'начать', u'должный', u'новый', u'день',
       u'метр', u'получить', u'далее', u'именно', u'апрель',
       u'сообщать', u'разный', u'говорить', u'делать',
       u'появиться', u'2016',
       u'2015', u'получить', u'иметь', u'составить', u'дать', u'читать',
       u'ничто', u'достаточно', u'использовать',
       u'принять', u'практически',
       u'находиться', u'месяц', u'достаточно', u'что-то', u'часто',
       u'хотеть', u'начаться', u'делать', u'событие', u'составлять',
       u'остаться', u'заявить', u'сделать', u'дело',
       u'примерно', u'попасть', u'хотя', u'лишь', u'первое',
       u'больший', u'решить', u'число', u'идти', u'давать', u'вопрос',
       u'сегодня', u'часть', u'высокий', u'главный', u'случай', u'место',
       u'конец', u'работать', u'работа', u'слово', u'важный', u'сказать']
)

In [88]:
url_start = 'http[s]?://'
url_end = (
    '(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
)
pattern = url_start + url_end

In [89]:
group_clean_doc_dict = {}
counter = 0

for group_id, doc in group_doc_dict.items():
    soup = BeautifulSoup(doc, 'html.parser')
    body = ' '.join(
        [tag.string.replace('\\n', ' ').replace('\\r', ' ')
         for tag in soup.descendants if tag.string]
    )
    body = re.sub('\[.*?\]','', body)
    body = re.sub(pattern,'', body)
    if body != '':
        body_clean = body.translate(translation_table).lower().strip()
        words = word_tokenize(body_clean)
        tokens = []
        # stemming and text normalization
        for word in words:
            if re.match('^[a-z0-9-]+$', word) is not None:
                tokens.append(PortSt.stem(word))
            elif word.count('-') > 1:
                tokens.append(word)
            else:
                normal_forms = morph.normal_forms(word)
                tokens.append(normal_forms[0] if normal_forms else word)
        # remove stopwords and leave unique words only
        tokens = filter(
            lambda token: token not in stopw, sorted(set(tokens))
        )

        # remove all words with more than 3 chars
        tokens = filter(lambda token: len(token) > 3, tokens)
    else:
        tokens = []
    counter += 1
    if counter % 100 == 0:
        print("{0} docs processed".format(counter))
    group_clean_doc_dict[group_id] = tokens

100 docs processed
200 docs processed
300 docs processed
400 docs processed
500 docs processed
600 docs processed
700 docs processed
800 docs processed
900 docs processed
1000 docs processed
1100 docs processed
1200 docs processed
1300 docs processed
1400 docs processed
1500 docs processed
1600 docs processed
1700 docs processed
1800 docs processed
1900 docs processed
2000 docs processed
2100 docs processed
2200 docs processed
2300 docs processed


In [97]:
group_clean_doc_dict = {key: list(val) for key, val in group_clean_doc_dict.items()}

In [98]:
save_obj(group_clean_doc_dict, 'group_doc_dict_clean')

In [100]:
from gensim.corpora import TextCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

class ListTextCorpus(TextCorpus):

    def get_texts(self):
        for doc in self.input:
            yield doc
                
mycorp = ListTextCorpus(input=group_clean_doc_dict.values())
justlda = LdaModel(
    corpus=mycorp, num_topics=20, passes=12
)

In [103]:
print('LdaModel performance')
for i in range(20):
    terms = justlda.get_topic_terms(i)
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))

LdaModel performance
0 человек жизнь знать твой никто видеть хотеться почему глаз пожалуйста
1 ребёнок жизнь ждать сохранить человек красивый судьба твой друг любовь
2 друг конора рука победить макгрегор тренировка хабиб боец нурмагомед конором
3 dark rock roll идея night blue фото music следующий time
4 человек жить любовь 2018 возpаста невoзмoжнo жpaть ждать утро внешность
5 игрок матч лига пенальти состав победа серия клуб футболист выиграть
6 октябрь ждать игра коммент приходить крутой пройти друг любимый последний
7 человек жизнь друг группа фото смотреть видеть любимый момент клип
8 человек жизнь друг знать жить любить ждать видеть смотреть найти
9 цена капот кузов music нибыть продажа фильтр привод фара колодка
10 выглядеть милый любить человек писать друг красивый стиль настроение акция
11 человек жизнь друг знать ждать думать смотреть найти любить смочь
12 салат рецепт ингредиент вкусный яйцо куриный блюдо соль вкусно соус
13 кататься шлем индивидуальный вчера жизнь группа вкл

In [109]:
dummy_dict = {key: 0 for key in range(20)}

group_topics_dict_20 = {
    group_id: dict(list(dummy_dict.items()) + justlda.get_document_topics(mycorp.dictionary.doc2bow(text)))
    for group_id, text in group_clean_doc_dict.items()
}
check_pd_20 = pd.DataFrame.from_dict(group_topics_dict_20, orient='index')
check_pd_20.head(10)
print("Group distribution by the most relevant topic")
pd.Series.round(check_pd_20.idxmax(axis=1).value_counts() * 1. / len(check_pd_20), 2)

Group distribution by the most relevant topic


11    0.51
16    0.08
17    0.06
8     0.06
0     0.04
12    0.04
14    0.03
7     0.02
15    0.02
3     0.01
2     0.01
10    0.01
5     0.01
4     0.01
18    0.01
1     0.01
6     0.01
13    0.01
9     0.01
19    0.01
dtype: float64

In [110]:
# dump lda model to disk
justlda.save('ldamodel_20_topics')

In [115]:
print("The most typical groups for every topic")
for i in range(20):
    terms = justlda.get_topic_terms(i)
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))
    typical_groups = check_pd_20[i].sort_values(ascending=False).index[:10]
    for g in typical_groups:
        group_info = vk_get_response(
            'groups.getById', 'group_ids={0}&v=4.9&lang=ru'.format(g), access_token
        )
        print(group_info['response'][0]['name'] + ' ' + 'http://vk.com/club' + str(g))
        time.sleep(0.3)
    print()

The most typical groups for every topic
0 человек жизнь знать твой никто видеть хотеться почему глаз пожалуйста
Гречневая Кафка http://vk.com/club69864801
Юмор Шрёдингера http://vk.com/club58737741
最も好きな写真 http://vk.com/club2629371
LUXURIOUS LIFE ™ http://vk.com/club39748345
Гриффиндор | Рик и Морти | Симпсоны http://vk.com/club88674651
MEN'S GROUP http://vk.com/club39765355
Иллюзионист http://vk.com/club127090395
Практическая психология http://vk.com/club41538339
MMOBoom.ru http://vk.com/club48007933
ГАРАЖ | GARAGE http://vk.com/club74297965

1 ребёнок жизнь ждать сохранить человек красивый судьба твой друг любовь
Мужчина должен быть... http://vk.com/club36719060
Киномания Кино театр  Москва http://vk.com/club50571799
Школа кулинарии | Вкусные рецепты http://vk.com/club49694199
Здоровые дети - тут все о семье, маме, малыше http://vk.com/club50059253
Дрифт http://vk.com/club58168203
Малогабаритка http://vk.com/club141409293
E S Q U I R E http://vk.com/club64434153
Men's Job | MJB http:

Вселенная MARVEL|DC http://vk.com/club112940897
Ищу модель Москва http://vk.com/club164682776
Новости Саранск | Мордовия «Столица С» http://vk.com/club38801259
videosos http://vk.com/club56263398
Спортсмен http://vk.com/club137624961
EMPIRE | История Человечества http://vk.com/club61134563

15 чтoб жизнь тoлькo кoгдa тeбить дeнь этoт ecть мeнить oдин
У каждого своя история http://vk.com/club23834959
ФИЛЬМЫ УЖАСОВ http://vk.com/club30823579
Бизнес секреты http://vk.com/club43037456
Все лгут http://vk.com/club16598458
These Magic Moments http://vk.com/club101240674
ЯЖЛЕДИ! http://vk.com/club24260227
Дневник Богини http://vk.com/club26717401
Видео http://vk.com/club26090596
► Женские мысли и чувства http://vk.com/club108862336
История|History http://vk.com/club53782768

16 размер отличный кожа любой сантиметр форма помочь цвета люба цвет
TrandyHall Шоу рум Одежда В наличии и на заказ http://vk.com/club47746606
Слова Великих людей http://vk.com/club28981879
Саморазвитие http://vk.com/club3

In [114]:
from operator import add

def groups_to_topics_v1(groups_list, n=5):
    counter = 1
    probs_sum = dummy_dict.values()
    for group_id in groups_list:
        topics_dict = group_topics_dict_20.get(group_id)
        rel_topic_probs = map(
            lambda x: x * (min(5, n) - counter + 1),
            dummy_dict.values() if not topics_dict else topics_dict.values()
        )
        probs_sum = map(add, probs_sum, rel_topic_probs)
        if counter == 5:
            break
        counter += 1
    return map(lambda x: x / sum([x + 1 for x in range(counter)]), probs_sum)

def groups_to_topics_v2(groups_list, n=5):
    counter = 1
    probs_sum = dummy_dict.values()
    for group_id in groups_list:
        topics_dict = group_topics_dict_20.get(group_id)
        rel_topic_probs = map(
            lambda x: x * 1,
            dummy_dict.values() if not topics_dict else topics_dict.values()
        )
        probs_sum = map(add, probs_sum, rel_topic_probs)
        if counter == 5:
            break
        counter += 1
    return map(lambda x: x, probs_sum)

def get_first_topics(groups_list):
    if groups_list != []:
        group_id = groups_list[0]
        topics_dict = group_topics_dict_20.get(group_id)
    else:
        topics_dict = dummy_dict
    return dummy_dict.values() if not topics_dict else topics_dict.values()

def get_main_interest(topic_probs):
    return np.argmax(topic_probs)